# Análise de Dados Massivos da PGFN utilizando Apache Spark

Este notebook documenta o desenvolvimento de um pipeline de processamento de dados utilizando Apache Spark. O objetivo principal é manipular e analisar grandes volumes de dados provenientes da Procuradoria-Geral da Fazenda Nacional (PGFN). A análise foca em dados de Pessoas Jurídicas (PJ) e Pessoas Físicas (PF), com o intuito de identificar padrões e realizar agregações significativas que possam ser utilizadas para fins acadêmicos e institucionais.

O projeto faz parte de um estudo acadêmico no contexto do mestrado em Ciências de Dados do Programa de Pós-Graduação em Ciência da Computação da Universidade de Brasília (PPCA-UNB). Através deste pipeline, buscamos explorar técnicas de processamento paralelo e big data para lidar com desafios comuns em cenários de dados massivos.

### Objetivos Específicos:
1. **Importação e Configuração**: Configurar o ambiente Spark e importar os dados da PGFN armazenados no Google Cloud Storage.
2. **Transformações Iniciais**: Limpar e filtrar os dados para obter um conjunto de dados relevante para a análise.
3. **Agrupamento e Agregação**: Agrupar os dados por categorias importantes e calcular métricas como soma e média das dívidas.
4. **Persistência dos Resultados**: Salvar os resultados processados de volta no Google Cloud Storage para análises posteriores.

### Estrutura do Notebook:
1. **Configuração do Ambiente**: Importação das bibliotecas e configuração do Spark.
2. **Leitura dos Dados**: Leitura dos dados brutos do Google Cloud Storage.
3. **Pré-processamento**: Limpeza e filtragem dos dados.
4. **Análise de Dados**: Agrupamento, agregação e cálculo das métricas.
5. **Salvamento dos Resultados**: Persistência dos dados processados em formato CSV.

Este pipeline demonstra a eficácia do uso de Apache Spark para manipulação e análise de grandes volumes de dados, oferecendo insights valiosos sobre as dívidas registradas na PGFN. Esperamos que este estudo contribua para uma melhor compreensão e gerenciamento dos dados financeiros no Brasil.

In [44]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, regexp_extract, desc, concat, lit
from pyspark.sql import functions as F

## Configuração do ambiente

### Abertura de Sessão

In [8]:
# Criar uma sessão Spark ou reutilizar a existente
spark = SparkSession.builder \
    .appName("Leitura e União de Múltiplos CSVs da PGFN") \
    .getOrCreate()

In [57]:
# Obter o contexto Spark
sc = spark.sparkContext

# Verificar o número de núcleos totais no cluster
total_cores = sc._jsc.sc().getExecutorMemoryStatus().size()
print(f"Total number of cores in the cluster: {total_cores}")

# Verificar o número de núcleos disponíveis por executor
conf = sc.getConf()
executor_cores = conf.get("spark.executor.cores")
print(f"Number of cores per executor: {executor_cores}")

Total number of cores in the cluster: 2
Number of cores per executor: 1


Os arquivos já foram baixados, decompactados e feito `upload` no Google Cloud Storage. Permite uma interação mais transparente com os clusters do Google Cloud DataProc.

In [10]:
csv_directory = "gs://dados-massivos/extracted_files_pgfn"

## Coleta dos dados

Leitura e agrupamentos dos dados

In [11]:
df = spark.read.option("header", "true").option("sep", ";").option("encoding", "ISO-8859-1").csv(csv_directory)

# Mostrar as primeiras linhas do DataFrame
df.show()

# Verificar o schema do DataFrame
df.printSchema()

# Contar o número de linhas no DataFrame
row_count = df.count()
print(f"Total number of rows: {row_count}")

+------------------+---------------+--------------+--------------------+----------+-------------------+----------------+-----------------------+--------------------+--------------------+--------------+------------------+-----------------+
|          CPF_CNPJ|    TIPO_PESSOA|  TIPO_DEVEDOR|        NOME_DEVEDOR|UF_DEVEDOR|UNIDADE_RESPONSAVEL|NUMERO_INSCRICAO|TIPO_SITUACAO_INSCRICAO|  SITUACAO_INSCRICAO|   RECEITA_PRINCIPAL|DATA_INSCRICAO|INDICADOR_AJUIZADO|VALOR_CONSOLIDADO|
+------------------+---------------+--------------+--------------------+----------+-------------------+----------------+-----------------------+--------------------+--------------------+--------------+------------------+-----------------+
|      XXX735.623XX|  Pessoa física|     PRINCIPAL|JOSE ALEKSANDRO D...|        AC|               ACRE|   1010300352142|            Em cobrança|      ATIVA AJUIZADA|Receita da dívida...|    25/11/2003|               SIM|       1551965.40|
|      XXX333.272XX|  Pessoa física|     PRI

Total number of rows: 27398260


## Tratamento

### Apenas devedores principais

In [13]:
df_filtered = df.filter(df["TIPO_DEVEDOR"] == "PRINCIPAL")
df_filtered.count()

19833978

In [14]:
df_filtered.show(5)

+------------------+---------------+------------+--------------------+----------+-------------------+----------------+-----------------------+--------------------+--------------------+--------------+------------------+-----------------+
|          CPF_CNPJ|    TIPO_PESSOA|TIPO_DEVEDOR|        NOME_DEVEDOR|UF_DEVEDOR|UNIDADE_RESPONSAVEL|NUMERO_INSCRICAO|TIPO_SITUACAO_INSCRICAO|  SITUACAO_INSCRICAO|   RECEITA_PRINCIPAL|DATA_INSCRICAO|INDICADOR_AJUIZADO|VALOR_CONSOLIDADO|
+------------------+---------------+------------+--------------------+----------+-------------------+----------------+-----------------------+--------------------+--------------------+--------------+------------------+-----------------+
|      XXX735.623XX|  Pessoa física|   PRINCIPAL|JOSE ALEKSANDRO D...|        AC|               ACRE|   1010300352142|            Em cobrança|      ATIVA AJUIZADA|Receita da dívida...|    25/11/2003|               SIM|       1551965.40|
|      XXX333.272XX|  Pessoa física|   PRINCIPAL|SAN

### Tipos de personalidades dos devedores

In [15]:
unique_tipos_pessoa = df.select("TIPO_PESSOA").distinct()
unique_tipos_pessoa.show()

+---------------+
|    TIPO_PESSOA|
+---------------+
|Pessoa jurídica|
|  Pessoa física|
+---------------+



## Apenas PF

In [16]:
df_pf = df_filtered.filter(df_filtered["TIPO_PESSOA"] == "Pessoa física")

In [17]:
num_rows = df_pf.count()
num_cols = len(df_pf.columns)

print(f"Shape: ({num_rows}, {num_cols})")

Shape: (2976066, 13)


### Variáveis de interesse

In [19]:
df_pf = df_pf.select("CPF_CNPJ", "NOME_DEVEDOR", "VALOR_CONSOLIDADO")

In [20]:
df_pf.show(5)

+------------+--------------------+-----------------+
|    CPF_CNPJ|        NOME_DEVEDOR|VALOR_CONSOLIDADO|
+------------+--------------------+-----------------+
|XXX735.623XX|JOSE ALEKSANDRO D...|       1551965.40|
|XXX333.272XX|SANDRA MONICA DAS...|           316.83|
|XXX735.623XX|JOSE ALEKSANDRO D...|          2932.06|
|XXX469.803XX|JORGE LUIS ALVES ...|         16459.34|
|XXX652.523XX|LUZILENA DE SOUSA...|          7691.42|
+------------+--------------------+-----------------+
only showing top 5 rows



### Transformações de atributos

Manter o mesmo comportamento das informações da Receita Federal do Brasil

In [22]:
# Substituir todas as ocorrências de "X" por "*" e remover os pontos na coluna "CPF_CNPJ"
df_pf = df_pf.withColumn("CPF_CNPJ", regexp_replace("CPF_CNPJ", "X", "*"))
df_pf = df_pf.withColumn("CPF_CNPJ", regexp_replace("CPF_CNPJ", r"\.", ""))

# Mostrar as primeiras linhas do DataFrame resultante para verificação
df_pf.show()

+-----------+--------------------+-----------------+
|   CPF_CNPJ|        NOME_DEVEDOR|VALOR_CONSOLIDADO|
+-----------+--------------------+-----------------+
|***735623**|JOSE ALEKSANDRO D...|       1551965.40|
|***333272**|SANDRA MONICA DAS...|           316.83|
|***735623**|JOSE ALEKSANDRO D...|          2932.06|
|***469803**|JORGE LUIS ALVES ...|         16459.34|
|***652523**|LUZILENA DE SOUSA...|          7691.42|
|***298509**|JOAO BATISTA DA S...|         17360.48|
|***946205**|AIDIL ALCOERES CO...|          8925.97|
|***627041**|FIDELIS DAMASCENO...|         21299.49|
|***597201**|ANALIA DIONISIA P...|          2980.44|
|***097916**|JULIO CEZAR DE OL...|          1523.17|
|***127901**|HELAINE ANNITA TI...|         62839.60|
|***888215**|RENAN RAMOS DO NA...|          7309.34|
|***888215**|RENAN RAMOS DO NA...|          8097.98|
|***965325**|JOSE JADER LEITE ...|        960613.76|
|***069815**|ROBERVAL ARAUJO V...|         27257.49|
|***298781**|JOSE EDMILSON PER...|         147

In [23]:
num_rows = df_pf.count()
num_cols = len(df_pf.columns)

print(f"Shape: ({num_rows}, {num_cols})")

Shape: (2976066, 3)


### Agregação pelo somatório da dívida

In [32]:
# Agrupar por "NOME_DEVEDOR" e calcular "QUANTIDADE" e "SOMA"
df_agrupado_pf = df_pf.groupBy("NOME_DEVEDOR").agg(
    F.first("CPF_CNPJ").alias("CPF_CNPJ"),
    F.count("NOME_DEVEDOR").alias("QUANTIDADE"),
    F.sum("VALOR_CONSOLIDADO").alias("SOMA")
)

# Selecionar as colunas desejadas
df_agrupado_pf = df_agrupado_pf.select("CPF_CNPJ", "NOME_DEVEDOR", "QUANTIDADE", "SOMA")

# Mostrar as primeiras linhas do DataFrame resultante para verificação
df_agrupado_pf.show()

+-----------+--------------------+----------+------------------+
|   CPF_CNPJ|        NOME_DEVEDOR|QUANTIDADE|              SOMA|
+-----------+--------------------+----------+------------------+
|***018455**|AAB BENAIA SAMI N...|         2|10478.130000000001|
|***224453**|       AARAO BARBOSA|         3|24580.299999999996|
|***617964**|AARAO DAMASCENO N...|         1|            813.16|
|***876657**|AARAO DOS REIS PA...|         1|          63166.54|
|***707036**|AARAO FELISBERTO ...|         1|          17878.91|
|***787854**|AARON MARQUES DE ...|         4|60139.689999999995|
|***877261**| ABADIA ARAUJO SILVA|         2|24333.769999999997|
|***363601**|ABADIA CAMPOS AMARAL|         1|        4843881.91|
|***909386**|ABADIA DE FATIMA ...|         2|           7867.85|
|***772871**|ABADIA DE OLIVEIR...|         1|           2784.25|
|***841121**|  ABADIA DIVINA LIMA|         4|         166678.83|
|***698651**|ABADIA IVONI TEODORO|         1|           1395.77|
|***297471**|ABADIA PEREI

Persistencia

In [33]:
df_agrupado_pf.coalesce(1).write.mode("overwrite").option("header", "true").csv("gs://dados-massivos/dados-finais/df_agrupado_pf_pgfn")

### Top 20 mais devedores

In [59]:
top_20_quantidade = df_agrupado_pf.orderBy(desc("QUANTIDADE")).limit(20)
top_20_soma = df_agrupado_pf.orderBy(desc("SOMA")).limit(20)

#### Por quantidade de dívidas

In [35]:
top_20_quantidade.show()

+-----------+--------------------+----------+--------------------+
|   CPF_CNPJ|        NOME_DEVEDOR|QUANTIDADE|                SOMA|
+-----------+--------------------+----------+--------------------+
|***326788**| JORGE RUDNEY ATALLA|       746| 4.914649628999999E7|
|***104187**|      TORMAR PEREIRA|       678|   1316129.139999994|
|***423095**| MARIA JOSE DA SILVA|       592|          7870582.75|
|***380768**|JOSE ALBERTO DE LUCA|       553|  1289595.2899999996|
|***189281**|JOSE CARLOS DA SILVA|       550|1.6455988770000001E7|
|***541365**|JOSE CARLOS DOS S...|       404|   6346797.530000001|
|***672561**|MARIA APARECIDA D...|       398|   4499959.390000001|
|***298509**|JOAO BATISTA DA S...|       372|           7313910.5|
|***208321**|RICARDO FERREIRA ...|       355|   608699.7499999993|
|***174998**|  ARSENIO DE GOUVEIA|       330|   726607.4599999995|
|***114503**|LUIZ CARLOS DA SILVA|       326|1.4985419799999995E7|
|***836851**|JOSE ANTONIO DA S...|       307|          7152928

#### Por somatóros de dívidas

In [60]:
top_20_soma.show(10)

+-----------+--------------------+----------+--------------------+
|   CPF_CNPJ|        NOME_DEVEDOR|QUANTIDADE|                SOMA|
+-----------+--------------------+----------+--------------------+
|***947290**|MARCOS AURELIO CA...|         1|     6.92311474938E9|
|***673601**|MARCIO DO CARMO A...|         6| 4.676691893190001E9|
|***077908**|       CLEMENTE DANA|         3|     2.32837482318E9|
|***566491**|APARECIDA PAXECO ...|        13|1.8195661277299998E9|
|***848748**|     THOMEZIO CHELLI|         5|1.4957876283799999E9|
|***875738**|GETULIO MARTINS D...|        16|     1.19419727672E9|
|***755157**|CRISTIANA MARINI ...|         1|     1.09490046573E9|
|***887628**|LUIZ AUGUSTO FALA...|         6|1.0861248378400002E9|
|***811152**|JAILTON SANCHES D...|        19| 7.191101502900001E8|
|***896606**|  JAIR AFONSO LISBOA|         7|      6.6468221479E8|
+-----------+--------------------+----------+--------------------+
only showing top 10 rows



Persistencia

In [61]:
top_20_soma.coalesce(1).write.mode("overwrite").option("header", "true").csv("gs://dados-massivos/dados-finais/df_top_20_soma_pgfn")

## Apenas PJ

In [37]:
df_pj = df_filtered.filter(df_filtered["TIPO_PESSOA"] == "Pessoa jurídica")

In [38]:
num_rows = df_pj.count()
num_cols = len(df_pj.columns)

print(f"Shape: ({num_rows}, {num_cols})")

Shape: (16857912, 13)


### Variáveis de interesse

In [39]:
df_pj = df_pj.select("CPF_CNPJ", "NOME_DEVEDOR", "VALOR_CONSOLIDADO")

In [40]:
df_pj.show(5)

+------------------+--------------------+-----------------+
|          CPF_CNPJ|        NOME_DEVEDOR|VALOR_CONSOLIDADO|
+------------------+--------------------+-----------------+
|28.196.878/0001-63|MORESCHI CLINICA ...|           582.34|
|84.461.748/0001-81|ACBR COMPUTADORES...|        128977.24|
|84.661.206/0001-52|CAPA CONSTRUCOES ...|         55507.58|
|28.196.878/0001-63|MORESCHI CLINICA ...|          2824.28|
|84.461.748/0001-81|ACBR COMPUTADORES...|       3924561.31|
+------------------+--------------------+-----------------+
only showing top 5 rows



### Transformações de atributos

Manter o mesmo comportamento das informações da Receita Federal do Brasil

In [42]:
df_pj = df_pj.withColumn("CPF_CNPJ", regexp_replace("CPF_CNPJ", r"\.", ""))
df_pj = df_pj.withColumn("CPF_CNPJ", regexp_extract("CPF_CNPJ", r"(\d+)", 1))
df_pj.show()

+--------+--------------------+-----------------+
|CPF_CNPJ|        NOME_DEVEDOR|VALOR_CONSOLIDADO|
+--------+--------------------+-----------------+
|28196878|MORESCHI CLINICA ...|           582.34|
|84461748|ACBR COMPUTADORES...|        128977.24|
|84661206|CAPA CONSTRUCOES ...|         55507.58|
|28196878|MORESCHI CLINICA ...|          2824.28|
|84461748|ACBR COMPUTADORES...|       3924561.31|
|14104972|RJ FONTES CONSULT...|         22326.69|
|84461748|ACBR COMPUTADORES...|        559118.98|
|28196878|MORESCHI CLINICA ...|          4707.15|
|84461748|ACBR COMPUTADORES...|      14083388.13|
|14104972|RJ FONTES CONSULT...|         39190.52|
|84661206|CAPA CONSTRUCOES ...|        175192.96|
|84461748|ACBR COMPUTADORES...|      14415068.11|
|28196878|MORESCHI CLINICA ...|          1499.60|
|84461748|ACBR COMPUTADORES...|        396853.16|
|17767143|EKTEC INDUSTRIA E...|          1384.40|
|17767143|EKTEC INDUSTRIA E...|          1669.07|
|60643228|  FIBRIA CELULOSE SA|     115114009.22|


In [45]:
num_rows = df_pj.count()
num_cols = len(df_pj.columns)

print(f"Shape: ({num_rows}, {num_cols})")

Shape: (16857912, 3)


In [46]:
df_pj = df_pj.withColumn("NOME_DEVEDOR", concat(df_pj["NOME_DEVEDOR"], lit("_"), df_pj["CPF_CNPJ"]))

In [47]:
num_rows = df_pj.count()
num_cols = len(df_pj.columns)

print(f"Shape: ({num_rows}, {num_cols})")

Shape: (16857912, 3)


In [48]:
df_pj.show()

+--------+--------------------+-----------------+
|CPF_CNPJ|        NOME_DEVEDOR|VALOR_CONSOLIDADO|
+--------+--------------------+-----------------+
|28196878|MORESCHI CLINICA ...|           582.34|
|84461748|ACBR COMPUTADORES...|        128977.24|
|84661206|CAPA CONSTRUCOES ...|         55507.58|
|28196878|MORESCHI CLINICA ...|          2824.28|
|84461748|ACBR COMPUTADORES...|       3924561.31|
|14104972|RJ FONTES CONSULT...|         22326.69|
|84461748|ACBR COMPUTADORES...|        559118.98|
|28196878|MORESCHI CLINICA ...|          4707.15|
|84461748|ACBR COMPUTADORES...|      14083388.13|
|14104972|RJ FONTES CONSULT...|         39190.52|
|84661206|CAPA CONSTRUCOES ...|        175192.96|
|84461748|ACBR COMPUTADORES...|      14415068.11|
|28196878|MORESCHI CLINICA ...|          1499.60|
|84461748|ACBR COMPUTADORES...|        396853.16|
|17767143|EKTEC INDUSTRIA E...|          1384.40|
|17767143|EKTEC INDUSTRIA E...|          1669.07|
|60643228|FIBRIA CELULOSE S...|     115114009.22|


### Agregação pelo somatório da dívida

In [49]:
# Agrupar por "NOME_DEVEDOR" e calcular "QUANTIDADE" e "SOMA"
df_agrupado_pj = df_pj.groupBy("NOME_DEVEDOR").agg(
    F.first("CPF_CNPJ").alias("CPF_CNPJ"),
    F.count("NOME_DEVEDOR").alias("QUANTIDADE"),
    F.sum("VALOR_CONSOLIDADO").alias("SOMA")
)

# Selecionar as colunas desejadas
df_agrupado_pj = df_agrupado_pj.select("CPF_CNPJ", "NOME_DEVEDOR", "QUANTIDADE", "SOMA")

# Mostrar as primeiras linhas do DataFrame resultante para verificação
df_agrupado_pj.show()

+--------+--------------------+----------+------------------+
|CPF_CNPJ|        NOME_DEVEDOR|QUANTIDADE|              SOMA|
+--------+--------------------+----------+------------------+
|43982209|& MANEIRO CONSTRU...|         1|           3062.16|
|14642606|.B PRODUCOES E EV...|         7|         146727.88|
|17929574|.BISTRO IMPORTACA...|         1|          26571.17|
|11365521|.COM EMPREENDIMEN...|         3|          32953.08|
|23847141|.N.P. PZZARIA LAN...|         2|           5186.14|
|00138058|00.138.058 FRANCI...|         1|           3108.97|
|00149041|00.149.041 LTDA_0...|        21|         982811.74|
|00203405|00.203.405 LUIZ E...|         2| 5867.280000000001|
|00210599|00.210.599 ALESSA...|         1|           2878.19|
|00278314|00.278.314 FERNAN...|         1|           4941.28|
|00296128|00.296.128 JOSE L...|         2|           3006.61|
|00297635|00.297.635 LTDA_0...|         2|          51079.57|
|00301347|00.301.347 ROSILE...|         1|            1797.4|
|0042495

Persistencia

In [55]:
df_agrupado_pj.coalesce(1).write.mode("overwrite").option("header", "true").csv("gs://dados-massivos/dados-finais/df_agrupado_pj_pgfn")

In [50]:
num_rows = df_pj.count()
num_cols = len(df_pj.columns)

print(f"Shape: ({num_rows}, {num_cols})")

Shape: (16857912, 3)


### Top 20 mais devedores

In [62]:
top_20_quantidade = df_agrupado_pj.orderBy(desc("QUANTIDADE")).limit(20)
top_20_soma = df_agrupado_pj.orderBy(desc("SOMA")).limit(20)

#### Por quantidade de dívidas

In [52]:
top_20_quantidade.show()

+--------+--------------------+----------+--------------------+
|CPF_CNPJ|        NOME_DEVEDOR|QUANTIDADE|                SOMA|
+--------+--------------------+----------+--------------------+
|61583860|JDC ALPHA EMPREEN...|      3572|1.5583168209999973E8|
|10988814|IMOBILIARIA BELEM...|      2109|  4826920.4600000335|
|23637366|POTY SHOPPING SA_...|      2017|    7654289.99000003|
|03507375|IMOBILIARIA CASTR...|      1677|          4944273.08|
|07242894|NORPLAN CONSTRUCA...|      1273|  3140867.6500000046|
|10810901|SOCIEDADE IMOBILI...|      1143|1.2427524080000028E7|
|92772821|S.A. VIACAO AEREA...|      1134| 5.796393736240001E9|
|10110492|SOMATEL  SOCIEDAD...|      1060|   2562446.900000002|
|05493234|IMOBILIARIA SAO M...|      1046|   4121309.029999996|
|17073865|ALFACON  CONSTRUC...|      1028|   6720617.919999998|
|12274379|LAGINHA AGRO INDU...|      1008|1.0036298363599988E9|
|02522287|COUROADA INDUSTRI...|       967| 3.492703128300001E8|
|90063470|CONPASUL CONSTRUC...|       94

#### Por somatóros de dívidas

In [63]:
top_20_soma.show()

+--------+--------------------+----------+--------------------+
|CPF_CNPJ|        NOME_DEVEDOR|QUANTIDADE|                SOMA|
+--------+--------------------+----------+--------------------+
|33000167|PETROLEO BRASILEI...|       168|   8.235141670133E10|
|33592510|  VALE S.A._33592510|       154|4.660885669412001E10|
|44764595|CARITAL BRASIL LT...|        36|   3.226804819959E10|
|33042730|COMPANHIA SIDERUR...|        55|1.323178296434999...|
|58910316|INDUSTRIAS DE PAP...|       273|1.244713610438000...|
|01597168|TINTO HOLDING LTD...|       781|   1.042069550822E10|
|03300070|PPL PARTICIPACOES...|        37| 9.818719964099998E9|
|50469303|DUAGRO ADMINISTRA...|        19| 8.440093032559999E9|
|60703923|VIACAO AEREA SAO ...|       337|     8.03758798361E9|
|16628281|SAMARCO MINERACAO...|        37| 7.264096068199999E9|
|60746948|BANCO BRADESCO S....|       132| 7.225473557650001E9|
|07526557| AMBEV S.A._07526557|       215|     6.93536381366E9|
|61226890|BASE ENGENHARIA E...|       29

In [64]:
top_20_soma.coalesce(1).write.mode("overwrite").option("header", "true").csv("gs://dados-massivos/dados-finais/df_top_20_soma_pj_pgfn")

## Agrupando as informações

In [54]:
df_pgfb = df_agrupado_pj.unionByName(df_agrupado_pf)
df_pgfb.show()

+--------+--------------------+----------+------------------+
|CPF_CNPJ|        NOME_DEVEDOR|QUANTIDADE|              SOMA|
+--------+--------------------+----------+------------------+
|43982209|& MANEIRO CONSTRU...|         1|           3062.16|
|14642606|.B PRODUCOES E EV...|         7|         146727.88|
|17929574|.BISTRO IMPORTACA...|         1|          26571.17|
|11365521|.COM EMPREENDIMEN...|         3|          32953.08|
|23847141|.N.P. PZZARIA LAN...|         2|           5186.14|
|00138058|00.138.058 FRANCI...|         1|           3108.97|
|00149041|00.149.041 LTDA_0...|        21|         982811.74|
|00203405|00.203.405 LUIZ E...|         2| 5867.280000000001|
|00210599|00.210.599 ALESSA...|         1|           2878.19|
|00278314|00.278.314 FERNAN...|         1|           4941.28|
|00296128|00.296.128 JOSE L...|         2|           3006.61|
|00297635|00.297.635 LTDA_0...|         2|          51079.57|
|00301347|00.301.347 ROSILE...|         1|            1797.4|
|0042495

Persistindo

In [56]:
df_pgfb.coalesce(1).write.mode("overwrite").option("header", "true").csv("gs://dados-massivos/dados-finais/df_agrupado_geral_pgfn")